In [5]:

import numpy as np
import pandas as pd
from pathlib import Path
import gc


# xsuite
import xtrack as xt
import xmask as xm
import xfields as xf
import xpart as xp


import BBStudies.Tracking.XsuitePlus as xPlus
import BBStudies.Tracking.InteractionPoint as inp
import BBStudies.Physics.Detuning as tune
import BBStudies.Plotting.BBPlots as bbplt
import BBStudies.Physics.Base as phys
import BBStudies.Physics.Constants as cst

from IPython.display import clear_output




# Importing Collider and Twiss
#-------------------------------------
collider = xt.Multiline.from_json('colliders/collider_NO_OCTU.json')
twiss = {}
twiss['lhcb1'] = collider['lhcb1'].twiss().to_pandas()
twiss['lhcb2'] = collider['lhcb2'].twiss().reverse().to_pandas()
#-------------------------------------


# Filtering twiss to get rid of slices, entries and exits
#-------------------------------------
light_twiss = {}
for sequence in ['lhcb1','lhcb2']:
    light_twiss[sequence] = xPlus.filter_twiss(twiss[sequence].set_index('name'),entries=['drift','..','_entry','_exit']).reset_index()
#-------------------------------------

clear_output()

---
# Loading tracking
---

In [6]:
# loading_dict={
partition_name    = 'TEST'
partition_ID      = 'NO_OCTU'
data_path         = 'tracking/matching_test/DATA'
checkpoint_path   = 'tracking/matching_test/CHECKPOINTS'
data        = xPlus.Tracking_Interface.from_parquet(data_path         ,partition_name=partition_name,partition_ID=partition_ID)
_cpt        = xPlus.Tracking_Interface.from_parquet(checkpoint_path   ,partition_name=partition_name,partition_ID=partition_ID)
data._checkpoint = _cpt._checkpoint
data

╭───────────────────────────── Tracking_Interface ──────────────────────────────╮
│ ╭───────────────────────────────────────────────────────────────────────────╮ │
│ │ <BBStudies.Tracking.XsuitePlus.RenderingTracker object at 0x2af75cb8ef20> │ │
│ ╰───────────────────────────────────────────────────────────────────────────╯ │
│                                                                               │
│   context_name = 'ContextCupy'                                                │
│      exec_time = 792.7057473058812                                            │
│         method = '6d'                                                         │
│     monitor_at = 'tcp.c6l7.b1'                                                │
│        n_parts = 20000                                                        │
│        n_turns = 5000                                                         │
│       nemitt_x = 1.8e-06                                                      │
│       nemitt_y = 1.8e-06                                                      │
│    nemitt_zeta = 2.5189505667359247                                           │
│   parquet_data = '_data'                                                      │
│ particle_on_co = "<class 'xpart.particles.particles.Particles'>"              │
│   partition_ID = 'NO_OCTU'                                                    │
│ partition_name = 'TEST'                                                       │
│        sigma_z = 0.09                                                         │
│  start_at_turn = 0                                                            │
│   stop_at_turn = 5000                                                         │
╰───────────────────────────────────────────────────────────────────────────────╯

In [7]:
twiss_at_TCP = twiss['lhcb1'].set_index('name').loc[data.monitor_at]
print(f'TWISS (x,px): ({twiss_at_TCP.x:.3e},{twiss_at_TCP.px:.3e})')
print(f'COORD (x,px): ({data.coord.x.mean():.3e},{data.coord.px.mean():.3e})')

TWISS (x,px): (8.489e-07,-1.117e-08)
COORD (x,px): (-5.424e-07,-1.456e-09)
